# Working with data frames continued 
## Week 5 

Two very important concepts introduced this week around working with data frames are joins and reshaping your data structure. In this notebook we will look at some worked examples of both of these concepts in Python. 

In [2]:
# as always, first load packages and modules 
import pandas as pd 
import numpy as np

## Joins

In [3]:
# create a new dataframe called patient_details 
patient_details = pd.DataFrame.from_dict({"id": [1, 2, 3, 4, 5],           
                                          "age": [40, 56, 23, 45, 34]})

patient_details

,id,age
0,1,40
1,2,56
2,3,23
3,4,45
4,5,34


In [4]:
# create a new dataframe called appointment details 
## notice only IDs 3, 4, and 5 exist in both data sets 
appt_details = pd.DataFrame.from_dict({"id": [3, 4, 5, 6, 7],
                                       "month": ["March", "May", "June", "June", "April"],
                                       "return_visit": [True, False, True, True, False]})

appt_details

,id,month,return_visit
0,3,March,True
1,4,May,False
2,5,June,True
3,6,June,True
4,7,April,False


Now lets join these two data sets to create a data set for analysis. 

### Inner join 

In [5]:
# inner join 
analysis_inner = pd.merge(patient_details, appt_details, 
                          how = "inner", 
                          on = "id")

analysis_inner

,id,age,month,return_visit
0,3,23,March,True
1,4,45,May,False
2,5,34,June,True


After an inner join, only IDs `3`, `4`, and `5` are returned as they are the only matching rows in both data frames. 

### Full join 

In [6]:
# full join 
analysis_full = pd.merge(patient_details, appt_details, 
                         how = "outer", # full join is denoted with outer  
                         on = "id")

analysis_full

,id,age,month,return_visit
0,1,40.0,NaN,NaN
1,2,56.0,NaN,NaN
2,3,23.0,March,True
3,4,45.0,May,False
4,5,34.0,June,True
5,6,NaN,June,True
6,7,NaN,April,False


In a full join, as you can see, there are `NaN` values for IDs `1` and `2` in the `month` and `return_visit` columns as these IDs are not in the `appt_details` data frame. And there are `NaN` values for IDs `6` and `7` for in the `age` column as these IDs are not in the `patient_details` data frame. 

### Left join 

In [7]:
# left join 
analysis_left = pd.merge(patient_details, appt_details, 
                         how = "left", 
                         on = "id")

analysis_left 

,id,age,month,return_visit
0,1,40,NaN,NaN
1,2,56,NaN,NaN
2,3,23,March,True
3,4,45,May,False
4,5,34,June,True


In a left join, all of the data from `patient_details` was kept, resulting in `NaN` values for IDs `1` and `2` in the columns `month` and `return_visit` as these IDs are not in the `appt_details` data frame. 

### Right join 

In [8]:
# right join 
analysis_right = pd.merge(patient_details, appt_details,
                          how = "right", 
                          on = "id")

analysis_right 

,id,age,month,return_visit
0,3,23.0,March,True
1,4,45.0,May,False
2,5,34.0,June,True
3,6,NaN,June,True
4,7,NaN,April,False


A right join is essentially the opposite of a left join, keeping all of the data from `appt_details`. Thus, IDs `6` and `7` have `NaN` for `age` as these IDs are not in the `patient_details` data frame. 

If you switch which data frames are on the right and left, a right join and left join will produce the same result, just with columns in a different order. 

In [9]:
# left join with order of data frame inputs switched is the same as the right join above 
analysis_left_switch = pd.merge(appt_details, patient_details, 
                                how = "left", 
                                on = "id")

analysis_left_switch 
# which is the same as analysis_right only age is the last column rather than the 2nd

,id,month,return_visit,age
0,3,March,True,23.0
1,4,May,False,45.0
2,5,June,True,34.0
3,6,June,True,NaN
4,7,April,False,NaN


## But what about when the linkage keys are not called the same thing? 

In this case, we need to specify the arguments `left_on` for the left-hand side dataset and `right_on` for the right-hand side dataset. Let's look at a full join as an example. 

In [10]:
# reminder of what the columns are called in appt details 
appt_details 

,id,month,return_visit
0,3,March,True
1,4,May,False
2,5,June,True
3,6,June,True
4,7,April,False


In [11]:
# create a 2nd patient details df with a different name for the id variable 
patient_details2 = pd.DataFrame.from_dict({"patient": [1, 2, 3, 4, 5],           
                                          "age": [40, 56, 23, 45, 34]})

patient_details2

,patient,age
0,1,40
1,2,56
2,3,23
3,4,45
4,5,34


In [12]:
analysis_full2 = pd.merge(patient_details2, appt_details, 
                          how = "outer", 
                          left_on = "patient", 
                          right_on = "id")

analysis_full2

,patient,age,id,month,return_visit
0,1.0,40.0,NaN,NaN,NaN
1,2.0,56.0,NaN,NaN,NaN
2,3.0,23.0,3.0,March,True
3,4.0,45.0,4.0,May,False
4,5.0,34.0,5.0,June,True
5,NaN,NaN,6.0,June,True
6,NaN,NaN,7.0,April,False


In [13]:
analysis_full

,id,age,month,return_visit
0,1,40.0,NaN,NaN
1,2,56.0,NaN,NaN
2,3,23.0,March,True
3,4,45.0,May,False
4,5,34.0,June,True
5,6,NaN,June,True
6,7,NaN,April,False


As you can see, `analysis_full2` and `analysis_full` are the same in content, though when using `left_on` and `right_on`, both original data frame linked key columns are retained. If you prefer to have the merged data frame with a single ID or linking key column, you can change the name of the variables in the data frames to be merged to match first then perform your merge as above. 

In [14]:
patient_details2.rename({"patient": "id"}, 
                        axis = 1).merge(appt_details, 
                                        how = "outer", 
                                        on = "id")

# this does not rename patient to id in the patient_details2 df, it just does so for the merge operation
# notice we pass a dictionary structure to the rename method

,id,age,month,return_visit
0,1,40.0,NaN,NaN
1,2,56.0,NaN,NaN
2,3,23.0,March,True
3,4,45.0,May,False
4,5,34.0,June,True
5,6,NaN,June,True
6,7,NaN,April,False


In [15]:
# patient_details2 still has ID column called patient
## becuase in the code above we did not change the stored data object 
patient_details2.head()

,patient,age
0,1,40
1,2,56
2,3,23
3,4,45
4,5,34


To merge by multiple keys, you just need to pass the keys in a list to `pd.merge`. 
So for example: `pd.merge(df1, df2, how = "left", on = ["A", "B"])`

## Reshaping data

To move between wide and long data formats in `pandas` we use the `pd.pivot()` and `pd.melt()` functions. 

* `pd.pivot(df, index = , columns = , values =)`: reshape the data from long to wide, where 
    - `columns` are the columns used to make the new data frame's columns 
    - `values` are the columns to use for populating the new data frame's values 
    - `index` is the column(s) used to make the new data frame's index. If not specified, uses the existing index by default 
    
* `pd.melt(df, id_vars = , value_vars = , var_name = , value_name = )`: reshape the data from wide to long, where 
    - `id_vars` are the columns used as identifier variables 
    - `value_vars` are the column(s) to unpivot or make longer
    - `var_name` is the name used for the `variable` column
    - `value_name` is the name used for the `value` column 
    - `ignore_index` if `True`, original index is ignored. If `False`, the original index is retained

To strengthen our understanding of wide and long formats of data, we will again return the `gapminder` data that we have seen a few times before.

In [8]:
# remember to change the file path to the data to suit your own set up 
gap_data = pd.read_csv("/Users/ben/Library/CloudStorage/OneDrive-UniversityofEdinburgh/Documents/Courses/Data Types and Structures in Python and R/Projects/Data-Types-and-Structures-in-Python-and-R/data/gapminder_data.csv")

gap_data

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


Sometimes, as with the  gapminder dataset, we have multiple types of observed data. It is somewhere in between the purely long and wide data formats:

* 3 "ID variables": `continent`, `country`, `year`
* 3 "observation variables": `pop` `lifeExp`, `gdpPercap`

It is  pretty common to have data in this format in most cases despite not having ALL observations in 1 column, since all 3 observation variables have different units. Depending on your question being asked of the data, this can be (and often is) considered a tidy format. But we can play with switching it to purely long and wide formats to show what that means (i.e., long would be 4 ID variables and 1 observation variable).


In [11]:
# make data longer
# notice we are using the list data structure to pass multiple columns to some arguments 
gap_data_long = gap_data.melt(id_vars = ["country", "continent", "year"],
                              value_vars = ["lifeExp", "pop", "gdpPercap"],
                              var_name = "Metric",
                              value_name = "Value")

gap_data_long

,country,continent,year,Metric,Value
0,Afghanistan,Asia,1952,lifeExp,28.801000
1,Afghanistan,Asia,1957,lifeExp,30.332000
2,Afghanistan,Asia,1962,lifeExp,31.997000
3,Afghanistan,Asia,1967,lifeExp,34.020000
4,Afghanistan,Asia,1972,lifeExp,36.088000
...,...,...,...,...,...
5107,Zimbabwe,Africa,1987,gdpPercap,706.157306
5108,Zimbabwe,Africa,1992,gdpPercap,693.420786
5109,Zimbabwe,Africa,1997,gdpPercap,792.449960
5110,Zimbabwe,Africa,2002,gdpPercap,672.038623


In [10]:
# you do not necessarily need to specify the value_vars parameter 
## but it is good practice to be explicit in your code 

# same outcome as above
gap_data.melt(id_vars = ["country", "continent", "year"],
              var_name = "Metric",
              value_name = "Value")

,country,continent,year,Metric,Value
0,Afghanistan,Asia,1952,lifeExp,28.801000
1,Afghanistan,Asia,1957,lifeExp,30.332000
2,Afghanistan,Asia,1962,lifeExp,31.997000
3,Afghanistan,Asia,1967,lifeExp,34.020000
4,Afghanistan,Asia,1972,lifeExp,36.088000
...,...,...,...,...,...
5107,Zimbabwe,Africa,1987,gdpPercap,706.157306
5108,Zimbabwe,Africa,1992,gdpPercap,693.420786
5109,Zimbabwe,Africa,1997,gdpPercap,792.449960
5110,Zimbabwe,Africa,2002,gdpPercap,672.038623


Using our new `gap_data_long` data frame, we can convert it back be more like the original `gap_data` format 


In [16]:
# wide 
gap_data_wide = gap_data_long.pivot(columns = "Metric",
                                    values = "Value",
                                    index = ["country", "continent", "year"]).reset_index()

gap_data_wide

Metric,country,continent,year,gdpPercap,lifeExp,pop
0,Afghanistan,Asia,1952,779.445314,28.801,8425333.0
1,Afghanistan,Asia,1957,820.853030,30.332,9240934.0
2,Afghanistan,Asia,1962,853.100710,31.997,10267083.0
3,Afghanistan,Asia,1967,836.197138,34.020,11537966.0
4,Afghanistan,Asia,1972,739.981106,36.088,13079460.0
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,706.157306,62.351,9216418.0
1700,Zimbabwe,Africa,1992,693.420786,60.377,10704340.0
1701,Zimbabwe,Africa,1997,792.449960,46.809,11404948.0
1702,Zimbabwe,Africa,2002,672.038623,39.989,11926563.0


In [17]:
# if we remove the reset_index, our resulting df will have a multilevel index
## which could indeed be what you are interested in! 
wide_mindex = gap_data_long.pivot(columns = "Metric",
                                  values = "Value",
                                  index = ["country", "continent", "year"])

wide_mindex

Metric                       gdpPercap  lifeExp         pop
country     continent year                                 
Afghanistan Asia      1952  779.445314   28.801   8425333.0
                      1957  820.853030   30.332   9240934.0
                      1962  853.100710   31.997  10267083.0
                      1967  836.197138   34.020  11537966.0
                      1972  739.981106   36.088  13079460.0
...                                ...      ...         ...
Zimbabwe    Africa    1987  706.157306   62.351   9216418.0
                      1992  693.420786   60.377  10704340.0
                      1997  792.449960   46.809  11404948.0
                      2002  672.038623   39.989  11926563.0
                      2007  469.709298   43.487  12311143.0

[1704 rows x 3 columns]

In [19]:
## lets compare the indexes from gap_data_wide and wide_mindex 
print(gap_data_wide.index) # range index 

RangeIndex(start=0, stop=1704, step=1)


In [20]:
print(wide_mindex.index) # multi index 

MultiIndex([('Afghanistan',   'Asia', 1952),
            ('Afghanistan',   'Asia', 1957),
            ('Afghanistan',   'Asia', 1962),
            ('Afghanistan',   'Asia', 1967),
            ('Afghanistan',   'Asia', 1972),
            ('Afghanistan',   'Asia', 1977),
            ('Afghanistan',   'Asia', 1982),
            ('Afghanistan',   'Asia', 1987),
            ('Afghanistan',   'Asia', 1992),
            ('Afghanistan',   'Asia', 1997),
            ...
            (   'Zimbabwe', 'Africa', 1962),
            (   'Zimbabwe', 'Africa', 1967),
            (   'Zimbabwe', 'Africa', 1972),
            (   'Zimbabwe', 'Africa', 1977),
            (   'Zimbabwe', 'Africa', 1982),
            (   'Zimbabwe', 'Africa', 1987),
            (   'Zimbabwe', 'Africa', 1992),
            (   'Zimbabwe', 'Africa', 1997),
            (   'Zimbabwe', 'Africa', 2002),
            (   'Zimbabwe', 'Africa', 2007)],
           names=['country', 'continent', 'year'], length=1704)


We could also make the data even wider by passing multiple columns to the `values` parameter.

In [21]:
# wider data 

## again without resetting the index we will auotmatically have a MultiIndex 
### with country and continent as we are passing a list to the index parameter 
gap_data_wide2 = gap_data.pivot(columns = "year",
                                values = ["lifeExp", "pop", "gdpPercap"],
                                index = ["country", "continent"])

gap_data_wide2

lifeExp                                          \
year                            1952    1957    1962    1967    1972    1977   
country            continent                                                   
Afghanistan        Asia       28.801  30.332  31.997  34.020  36.088  38.438   
Albania            Europe     55.230  59.280  64.820  66.220  67.690  68.930   
Algeria            Africa     43.077  45.685  48.303  51.407  54.518  58.014   
Angola             Africa     30.015  31.999  34.000  35.985  37.928  39.483   
Argentina          Americas   62.485  64.399  65.142  65.634  67.065  68.481   
...                              ...     ...     ...     ...     ...     ...   
Vietnam            Asia       40.412  42.887  45.363  47.838  50.254  55.764   
West Bank and Gaza Asia       43.160  45.671  48.127  51.631  56.532  60.765   
Yemen, Rep.        Asia       32.548  33.970  35.180  36.984  39.848  44.175   
Zambia             Africa     42.038  44.077  46.023  47.768  50.107  51.386   
Zimbabwe           Africa     48.451  50.469  52.358  53.995  55.635  57.674   

                                                              ...  \
year                            1982    1987    1992    1997  ...   
country            continent                                  ...   
Afghanistan        Asia       39.854  40.822  41.674  41.763  ...   
Albania            Europe     70.420  72.000  71.581  72.950  ...   
Algeria            Africa     61.368  65.799  67.744  69.152  ...   
Angola             Africa     39.942  39.906  40.647  40.963  ...   
Argentina          Americas   69.942  70.774  71.868  73.275  ...   
...                              ...     ...     ...     ...  ...   
Vietnam            Asia       58.816  62.820  67.662  70.672  ...   
West Bank and Gaza Asia       64.406  67.046  69.718  71.096  ...   
Yemen, Rep.        Asia       49.113  52.922  55.599  58.020  ...   
Zambia             Africa     51.821  50.821  46.100  40.238  ...   
Zimbabwe           Africa     60.363  62.351  60.377  46.809  ...   

                                gdpPercap                            \
year                                 1962         1967         1972   
country            continent                                          
Afghanistan        Asia        853.100710   836.197138   739.981106   
Albania            Europe     2312.888958  2760.196931  3313.422188   
Algeria            Africa     2550.816880  3246.991771  4182.663766   
Angola             Africa     4269.276742  5522.776375  5473.288005   
Argentina          Americas   7133.166023  8052.953021  9443.038526   
...                                   ...          ...          ...   
Vietnam            Asia        772.049160   637.123289   699.501644   
West Bank and Gaza Asia       2198.956312  2649.715007  3133.409277   
Yemen, Rep.        Asia        825.623201   862.442146  1265.047031   
Zambia             Africa     1452.725766  1777.077318  1773.498265   
Zimbabwe           Africa      527.272182   569.795071   799.362176   

                                                                      \
year                                  1977         1982         1987   
country            continent                                           
Afghanistan        Asia         786.113360   978.011439   852.395945   
Albania            Europe      3533.003910  3630.880722  3738.932735   
Algeria            Africa      4910.416756  5745.160213  5681.358539   
Angola             Africa      3008.647355  2756.953672  2430.208311   
Argentina          Americas   10079.026740  8997.897412  9139.671389   
...                                    ...          ...          ...   
Vietnam            Asia         713.537120   707.235786   820.799445   
West Bank and Gaza Asia        3682.831494  4336.032082  5107.197384   
Yemen, Rep.        Asia        1829.765177  1977.557010  1971.741538   
Zambia             Africa      1588.688299  1408.678565  1213.315116   
Zimbabwe  

Let's compare the dimensions of our tibbles: 

In [22]:
gap_data.shape # 1704 rows and 6 columns 

(1704, 6)

In [23]:
gap_data_long.shape  # 5112 rows by 5 columns 

(5112, 5)

In [24]:
gap_data_wide.shape # 1704 rows and 6 columns - same as gap_data as expected

(1704, 6)

In [25]:
wide_mindex.shape # 1704 rows by 3 columns - becuase we have a MultiIndex with 3 columns! (hence 3 less columns than gap_data_wide)

(1704, 3)

In [26]:
gap_data_wide2.shape # 142 rows and 36 columns
## would be 142 rows by 38 columns if we reset the index - try it out and see

(142, 36)

As we can see by the dimensions, `gap_data_long` is indeed skinnier (fewer columns) and longer (many more rows) than the other data frames and `gap_data_wide2` is indeed much wider (many more columns and fewer rows)

### Multi-level indexes 

When working with multi-index data frames, we cannot use `df.pivot()` or `df.melt()` but must rather employ the `df.unstack()` and `df.stack()` methods. 

* `df.unstack(level = -1, fill_value = None)` is the MultiIndex equivalent to `df.pivot()`
    - `level` is default, it is set to `-1` (i.e., the last level can be unstacked). If we pass the specified level, it will unstack those levels. 
    - `fill_value` are the values you wish to replace resulting `NaN` values with in the new data frame, by default set to `None` 


* `df.stack(level = -1 , dropna = True)` is the MultiIndex equivalent to `df.melt()`
    - `level` is level(s) to stack from the column axis onto the index axis, defined as one index or label, or a list of indices or labels. By default set to `-1` (i.e., the last level can be stacked)
    - `dropna` is used to manage the rows with resulting `NaN` values. Default is `True` which drops rows in the resulting Frame/Series with all missing values

In [27]:
# create a same example df with a multi-level index to better see this in action 
multicol1 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                       ('weight', 'pounds')])

df_multi_level_cols1 = pd.DataFrame([[1, 2], [2, 4]],
                                    index = ['cat', 'dog'],
                                    columns = multicol1)

df_multi_level_cols1

weight       
        kg pounds
cat      1      2
dog      2      4

In [29]:
# stack the data frame which has a multi-level column axis 
stack1 = df_multi_level_cols1.stack(future_stack = True)

stack1

weight
cat kg           1
    pounds       2
dog kg           2
    pounds       4

We can also unstack these columns with the sister function `df.unstack()`

In [30]:
# unstack 
unstack1 = stack1.unstack()

unstack1

weight       
        kg pounds
cat      1      2
dog      2      4

It is common to have missing values when stacking a dataframe with multi-level columns, as the stacked dataframe typically has more values than the original dataframe. Missing values are filled with `NaN`

In [31]:
multicol2 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                        ('height', 'm')])

df_multi_level_cols2 = pd.DataFrame([[1.0, 2.0], [3.0, 4.0]],
                                     index = ['cat', 'dog'],
                                     columns = multicol2)

df_multi_level_cols2

,weight,height
,kg,m
cat,1.0,2.0
dog,3.0,4.0


In [33]:
stack2 = df_multi_level_cols2.stack(future_stack = True)

stack2

weight  height
cat kg     1.0     NaN
    m      NaN     2.0
dog kg     3.0     NaN
    m      NaN     4.0

In [34]:
# unstack 
unstack2 = stack2.unstack()

unstack2

weight     height     
        kg   m     kg    m
cat    1.0 NaN    NaN  2.0
dog    3.0 NaN    NaN  4.0

In [36]:
# stack by column axis 0 instead of default -1 
stack3 = df_multi_level_cols2.stack(level = 0, future_stack = True)

stack3

kg    m
cat weight  1.0  NaN
    height  NaN  2.0
dog weight  3.0  NaN
    height  NaN  4.0

In [37]:
# unstack 
unstack3 = stack3.unstack()

unstack3

kg             m       
    weight height weight height
cat    1.0    NaN    NaN    2.0
dog    3.0    NaN    NaN    4.0

In [38]:
# compare to unstack3 above 
unstack3_2 = stack3.unstack(level = 0)

unstack3_2

kg         m     
        cat  dog  cat  dog
weight  1.0  3.0  NaN  NaN
height  NaN  NaN  2.0  4.0

---

## You did it! 🎉 

Well done for making it to the end of this notebook. If you have not done so yet, move to the RMarkdwon notebook next. 

⭐⭐⭐❓👣 Do not forget your 3 stars, a wish, and a step mini-diaries once you have completed the content for this week. 

---
*Dr. Brittany Blankinship (2024)*